In [ ]:
#############分别用随机森林、逻辑回归、MultinomialNB、SGD进行舆情学习#####
############样本来自爬虫爬取的数据##########
分析新闻正负情感值：
读取新闻并且进行文本处理和分词
建立模型 build_and_evaluate(X,y,classifier,outpath='model')
采用了gridsearch， 允许同时运训多种参数同时进行
也可以gridsearch = False 如果不需要调参数
根据precision 和 recall 值， 对比Multibinomial, logistic regression, SGD等。最终采用随机森林 模型
outpath 允许模型保存到本地
用随机森林模型，得出每个词的重要性，也就是feature importance
寻找对于模型预测最有决定性的文字：show_most_informative_features(model, text=None, n=20):
这个function只适用于pipline结构，也就是在训练模型的时候，需要吧gridsearch关掉训练出来的模型才能用
随机森里不可用
会产出哪些词是正面导向和负面导向以及向量

In [1]:


import numpy as np
import pickle
import logging
np.random.seed(1337)
import re
import codecs
import jieba
import math
import numpy as np
from itertools import product, count  
from heapq import nlargest  
from gensim.models import word2vec  
from sklearn.utils import shuffle
try:
    reload(sys)
    sys.setdefaultencoding('utf-8')
except:
    pass

In [2]:
# import json
# with open("pos.JSON", 'r') as fp:
#     pos_text = json.load( fp)
# with open("neg.JSON", 'r') as fp:
#     neg_text = json.load( fp)


In [3]:
def sentence_split(str_centence):
    list_ret = list()
    for s_str in str_centence.split('。'):
        if '?' in s_str:
            list_ret.extend(s_str.split('？'))
        elif '!' in s_str:
            list_ret.extend(s_str.split('！'))
        else:
            list_ret.append(s_str.strip())
    return list_ret

In [4]:
stopwordset = set()
with open('sa/jieba_dict/stopwords.txt','r',encoding = 'utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))

def seg_art_list(art_list):
    corpus = []
    for title, art in art_list.items():
        corpus.append(" ".join(seg_sent(art)))
    
    return corpus
        
output = open('word_seg.txt','w')
def seg_sent(art):
    r = '[’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+'  
    art = art.strip()
    art = re.sub(r, '', art)
    art = re.sub("[a-z]","",art)
    l = []

    seg_list = list(jieba.cut(art, cut_all = False))
    for word in seg_list:
        if word not in stopwordset and word != ' ' and word != "\n" and word != "\n\n":
                output.write(word + ' ')
                l.append(word)
    return l


In [5]:
##vectorizer = TfidfVectorizer(sublinear_tf=True,  stop_words='english')

import json
# with open("pos.JSON", 'r') as fp:
#     pos_text = json.load( fp)
# with open("neg.JSON", 'r') as fp:
#     neg_text = json.load( fp)

#    title['fasf']= content

with open("/Users/daichanglin/Desktop/data/pos0307.JSON", 'r') as fp:
    pos_text = json.load( fp)
with open("/Users/daichanglin/Desktop/data/neg0307.JSON", 'r') as fp:
    neg_text = json.load( fp)
     

    
# tl = u"平安银行(000001)年报点评：不良加大处置 零售加快转型"
# if(tl in pos_text):
#     print(pos_text[tl])
    
# # tl = "物美掌权，业务有望稳中有进"
# # if(tl in neg_text):
# #     print(neg_text[tl])
    
# print(len(pos_text.items()))
# print("add after")
# for title, art in new_pos_text.items():
#     print(title)
#     pos_text[title]=art
# print(len(pos_text.items()))
# tl = u"平安银行(000001)年报点评：不良加大处置 零售加快转型"
# if(tl in pos_text):
#     print(pos_text[tl])
    
# tl = "物美掌权，业务有望稳中有进"
# if(tl in neg_text):
#     print(neg_text[tl])

print(len(pos_text))
print(len(neg_text))

negfile = "/Users/daichanglin/Desktop/igoldenbeta/robotSVN/robot/robot-parent/python/data/negfactor.txt"
i=0
with open(negfile,'r',encoding = 'utf-8') as sw:
    for line in sw:
        neg_text["neg"+str(i)]=line
        i=i+1
        
posfile = "/Users/daichanglin/Desktop/igoldenbeta/robotSVN/robot/robot-parent/python/data/posfactor.txt"
i=0
with open(posfile,'r',encoding = 'utf-8') as sw:
    for line in sw:
        pos_text["pos"+str(i)]=line
        i=i+1
        
print(len(pos_text))
print(len(neg_text))

pos_tfidf = seg_art_list(pos_text)
neg_tfidf = seg_art_list(neg_text)





Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/nr/4rg65wr168q0jp4wjc2kkh100000gn/T/jieba.cache


2318
348
3755
2568


Loading model cost 1.390 seconds.
Prefix dict has been built succesfully.


In [6]:
with open("pos_tfidf0307.JSON", 'w') as fp:
    json.dump(pos_tfidf, fp)
with open("neg_tfidf0307.JSON", 'w') as fp:
    json.dump(neg_tfidf, fp)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics.scorer import make_scorer
from sklearn import linear_model
from sklearn import metrics
import operator
from time import time
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import train_test_split as tts
import timeit
from sklearn.model_selection import GridSearchCV

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
#96条正面，123 条负面
X = list(pos_tfidf) + list(neg_tfidf)
y = [1] * len(pos_tfidf) + [-1]*len(neg_tfidf)

In [9]:
print("finish")
print(len(pos_tfidf))
print(len(neg_tfidf))
print(len(X))
print(len(y))
# print(X[0:2])
# print(y[0:2])

# finish
# 2318
# 348
# 2666
# 2666


finish
3755
2568
6323
6323


### 模型训练 

In [10]:
# X_train, X_test, y_train, y_test
# iniData = False
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)
len(X_train)
# print(x)

def build_and_evaluate(X, y, classifier=SGDClassifier,outpath = False, grid_Search = True):

    def build(classifier, X, y=None):
        """
        Inner build function that builds a single model.
        """
        if isinstance(classifier, type):
            classifier = classifier()
        if grid_Search:
            parameters = {'vect__max_df': (0.5, 0.75, 1.0),
                        #'vect__max_features': (None, 5000, 10000, 50000),
                        'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
                        'tfidf__use_idf': (True, False),
                        'tfidf__norm': ('l1', 'l2'),
                        #'clf__alpha': (0.00001, 0.000001),
                        #'clf__penalty': ('l2', 'elasticnet'),
                         #'clf__n_iter': (10, 50, 80),
                }

            pipeline = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', classifier),])
            grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)
            print("Performing grid search...")
            print("pipeline:", [name for name, _ in pipeline.steps])
            print("parameters:")
            print(parameters)
            t0 = time()
            grid_search.fit(X, y)
            print("done in %0.3fs" % (time() - t0))
            print()

            print("Best score: %0.3f" % grid_search.best_score_)
            print("Best parameters set:")
            best_parameters = grid_search.best_estimator_.get_params()
            for param_name in sorted(parameters.keys()):
                print("\t%s: %r" % (param_name, best_parameters[param_name]))
            #model.fit(X, y)
            return grid_search
        else:
            pipeline = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),min_df=10,max_df = 0.9)),
                                 ('tfidf', TfidfTransformer()),('clf', classifier),])
            t0 = time()
            pipeline.fit(X, y)

            print("done in %0.3fs" % (time() - t0))
            return pipeline
    # Begin evaluation


    labels = LabelEncoder()

    y = labels.fit_transform(y)

    print("Building for evaluation")
#     X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)
#     if(iniData==False):
#         X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)
#         iniData = True

#     X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)
    model = build(classifier, X_train, y_train)

    y_pred = model.predict(X_test)
    print("classifier result:")
    print(clsr(y_test, y_pred, target_names=['neg', 'pos']))
    if outpath:
        with open(outpath, 'wb') as f:
            pickle.dump(model, f)

        print("Model written out to {}".format(outpath))
    print("confusion matrix")
    print(metrics.confusion_matrix(y_test, y_pred))

    return model


## 1. MultinomialNB

In [11]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB

model_Mb = build_and_evaluate(X,y,classifier,outpath='model_nb', grid_Search=False)

Building for evaluation
done in 0.953s
classifier result:
             precision    recall  f1-score   support

        neg       0.96      0.86      0.91       542
        pos       0.90      0.98      0.94       723

avg / total       0.93      0.93      0.93      1265

Model written out to model_nb
confusion matrix
[[466  76]
 [ 17 706]]


# 逻辑回归&信贷

## 2. Logistic Regression

In [12]:

from sklearn.linear_model import LogisticRegression
classifier= LogisticRegression
model_Lr = build_and_evaluate(X,y,classifier,outpath='model_lr',grid_Search=False)

Building for evaluation
check_X_y
[1, 1, -1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, -1, 1, 1, -1, 1, -1, 1, -1, -1, -1, -1, 1, 1, -1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, 1, 1, -1, -1, 1, -1, -1, 1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, 1, -1, -1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, -1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1

classifier result:
             precision    recall  f1-score   support

        neg       1.00      0.93      0.96       542
        pos       0.95      1.00      0.98       723

avg / total       0.97      0.97      0.97      1265

Model written out to model_lr
confusion matrix
[[506  36]
 [  1 722]]


In [13]:

from sklearn.linear_model import LogisticRegression
classifier= LogisticRegression

model_Lr = build_and_evaluate(X,y,classifier,outpath='model',grid_Search=False)

Building for evaluation
check_X_y
[1, 1, -1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, -1, 1, 1, -1, 1, -1, 1, -1, -1, -1, -1, 1, 1, -1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, 1, 1, -1, -1, 1, -1, -1, 1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, 1, -1, -1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, -1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1

classifier result:
             precision    recall  f1-score   support

        neg       1.00      0.93      0.96       542
        pos       0.95      1.00      0.98       723

avg / total       0.97      0.97      0.97      1265

Model written out to model
confusion matrix
[[506  36]
 [  1 722]]


## 3. Stochastic gradient descent

In [14]:
from sklearn.linear_model import SGDClassifier
classifier= SGDClassifier
model_default = build_and_evaluate(X,y,outpath='model_SGD',grid_Search=False)


Building for evaluation
done in 0.905s
classifier result:
             precision    recall  f1-score   support

        neg       0.99      0.98      0.98       542
        pos       0.98      0.99      0.99       723

avg / total       0.99      0.99      0.99      1265

Model written out to model_SGD
confusion matrix
[[530  12]
 [  5 718]]


## 4. Random Forest

In [15]:
from sklearn.ensemble import RandomForestClassifier

classifier= RandomForestClassifier(n_estimators=500,min_samples_leaf=2)

model_RF = build_and_evaluate(X,y,classifier,outpath='model0607_rf',grid_Search=False)

Building for evaluation
done in 8.563s
classifier result:
             precision    recall  f1-score   support

        neg       0.99      0.91      0.95       542
        pos       0.94      0.99      0.97       723

avg / total       0.96      0.96      0.96      1265

Model written out to model0607_rf
confusion matrix
[[495  47]
 [  5 718]]


In [16]:
print('负面\t\t','正面')
print(model_RF.predict_proba(neg_tfidf[10:20]))
print(model_RF.predict_proba(pos_tfidf[10:20]))

负面		 正面
[[ 0.81782449  0.18217551]
 [ 0.96708653  0.03291347]
 [ 0.83584196  0.16415804]
 [ 0.77410071  0.22589929]
 [ 0.88186226  0.11813774]
 [ 0.89648983  0.10351017]
 [ 0.91399837  0.08600163]
 [ 0.82079407  0.17920593]
 [ 0.73295685  0.26704315]
 [ 0.87347313  0.12652687]]
[[ 0.05964444  0.94035556]
 [ 0.1248453   0.8751547 ]
 [ 0.14195108  0.85804892]
 [ 0.17514497  0.82485503]
 [ 0.45551986  0.54448014]
 [ 0.09201905  0.90798095]
 [ 0.0281      0.9719    ]
 [ 0.03308658  0.96691342]
 [ 0.0306      0.9694    ]
 [ 0.12696136  0.87303864]]


In [17]:
model_Lr.predict_proba(neg_tfidf[10:20])

array([[ 0.99865237,  0.00134763],
       [ 0.94657015,  0.05342985],
       [ 0.99650224,  0.00349776],
       [ 0.93971509,  0.06028491],
       [ 0.81209   ,  0.18791   ],
       [ 0.88944153,  0.11055847],
       [ 0.99397411,  0.00602589],
       [ 0.81503642,  0.18496358],
       [ 0.60603066,  0.39396934],
       [ 0.94665958,  0.05334042]])

In [18]:
model_Mb.predict_proba(neg_tfidf[10:20])

array([[ 0.99660067,  0.00339933],
       [ 0.99376394,  0.00623606],
       [ 0.98004583,  0.01995417],
       [ 0.93325027,  0.06674973],
       [ 0.49872323,  0.50127677],
       [ 0.97111486,  0.02888514],
       [ 0.99844626,  0.00155374],
       [ 0.18555475,  0.81444525],
       [ 0.03020401,  0.96979599],
       [ 0.97748797,  0.02251203]])

## 几个模型的训练结果是随机森林效果最佳

### 最有决定性的文字：


#### 这个需要grid_searhch = False

In [19]:
def show_most_informative_features(model, text=None, n=100):
    # Extract the vectorizer and the classifier from the pipeline
    vectorizer = model.named_steps['vect']
    classifier = model.named_steps['clf']

    # Check to make sure that we can perform this computation
    if not hasattr(classifier, 'coef_'):
        raise TypeError(
            "Cannot compute most informative features on {}.".format(
                classifier.__class__.__name__
            )
        )

    if text is not None:
        # Compute the coefficients for the text
        tvec = model.transform([text]).toarray()
    else:
        # Otherwise simply use the coefficients
        tvec = classifier.coef_

    # Zip the feature names with the coefs and sort
    coefs = sorted(
        zip(tvec[0], vectorizer.get_feature_names()),
        key=operator.itemgetter(0), reverse=True
    )

    # Get the top n and bottom n coef, name pairs
    topn  = zip(coefs[:n], coefs[:-(n+1):-1])

    # Create the output string to return
    output = []

    # If text, add the predicted value to the output.
    if text is not None:
        output.append("\"{}\"".format(text))
        output.append(
            "Classified as: {}".format(model.predict([text]))
        )
        output.append("")

    # Create two columns with most negative and most positive features.
    for (cp, fnp), (cn, fnn) in topn:
        output.append(
            "{:0.4f}{: >15}    {:0.4f}{: >15}".format(
                cp, fnp, cn, fnn
            )
        )

    return "\n".join(output)


In [20]:
import pandas as pd
# import pandas
# classifier= RandomForestClassifier(n_estimators=20,random_state=5,min_samples_leaf=2)
# model_MRF = build_and_evaluate(X,y,classifier,outpath='model',grid_Search=False)
# #   pipeline = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', classifier),])
# tfidf = model_MRF.named_steps['tfidf']       
# feature_names = list(tfidf.get_feature_names())
        
# importances = pandas.DataFrame({'name':feature_names,'imp':model_MRF.feature_importances_
#                             }).sort(['imp'], ascending = False).reset_index(drop = True)

# importances.head(100)

# words = show_most_informative_features(model_MRF)
# print(words)


# tfidf =  model_RF.named_steps['tfidf']
vectorizer = model_RF.named_steps['vect']
classifier = model_RF.named_steps['clf']
feature_names = list(vectorizer.get_feature_names())


feature_importance = pd.DataFrame({'name':feature_names, 'importance':classifier.feature_importances_},columns = ['name','importance'])
sort_importance = feature_importance.sort_values(by= ['importance'],ascending = False)
print(sort_importance.head(500))

      name  importance
2238    增持    0.195328
4500    辞职    0.063002
1632    减持    0.052872
1562    关于    0.025342
2971    持股    0.015173
4290    计划    0.015059
1107    中性    0.014017
4819    预计    0.011886
2396    完成    0.011630
1301    以上    0.010797
4836    风险    0.010086
3028    提示    0.009226
4324    评级    0.008670
296   2017    0.008211
1274    亿元    0.008130
4813    预期    0.007906
1542  公司股票    0.007896
4068    维持    0.007820
2246    增长    0.007713
1660    分别    0.006716
2458    对应    0.006598
4552    进展    0.006478
932     一致    0.006351
2919    投资    0.006340
3766    监事    0.006307
4227    行动    0.006167
3024    提升    0.005882
4222    行业    0.005850
3264    有望    0.005619
2551    市场    0.005595
...    ...         ...
1929  去年同期    0.000142
1171    乐普    0.000142
827    asp    0.000142
2176    地位    0.000141
3303    本期    0.000141
2152    国家    0.000140
337     21    0.000140
86      08    0.000139
4569    迹象    0.000139
3251   有利于    0.000139
1109    中报    0.000138
909     一定 

In [21]:
words_Mb = show_most_informative_features(model_Mb)
print(words_Mb)

-4.1876             增持    -10.1457             遭到
-4.4160             关于    -10.1457             选举
-4.9259             计划    -10.1457             辞职
-5.1720             亿元    -10.1457             轮候
-5.1736             增长    -10.1457             补选
-5.3056             进展    -10.1457           职工代表
-5.5072             业务    -10.1457             冻结
-5.5137             同比    -10.1457           保留意见
-5.5322           2017    -10.1457         1q2017
-5.6019             完成    -10.1180             聘任
-5.7545             产品    -10.1054             平仓
-5.7657             实现    -10.1007             秘书
-5.7809            净利润    -10.0774         4q2016
-5.7810             业绩    -10.0628             逾期
-5.7866             预计    -10.0457             总裁
-5.8727             分别    -10.0408            监事会
-5.8955             有望    -10.0351             纠纷
-5.9051             市场    -10.0170             离职
-5.9097             行业    -10.0071             事务
-5.9114             项目    -9.9630             增补
-

In [22]:
words_Lr = show_most_informative_features(model_Lr)
print(words_Lr)

11.9572             增持    -8.0440             减持
3.0748           2017    -7.1368             辞职
2.5332             增长    -5.3061             中性
2.1194             有望    -2.9496             冻结
1.7797             预计    -2.5972           2016
1.7282           2018    -2.3951           2015
1.6469             项目    -2.2834             诉讼
1.6131             产品    -2.1651             平仓
1.5777             增速    -2.0196             离职
1.5124             国内    -1.9367             质押
1.4946             行业    -1.7758             监事
1.4790            三季度    -1.7279            上半年
1.4654             分别    -1.6985           1h17
1.4526           公司股票    -1.6979             涉及
1.4460             完成    -1.5688             下降
1.4293             提升    -1.5664       20162018
1.4024             持续    -1.4743             持股
1.3917             计划    -1.4130            毛利率
1.3508             万吨    -1.3998             独立
1.3394             龙头    -1.3904             处罚
1.2970             环比    -1.3609       

In [23]:
model_default = show_most_informative_features(model_default)
print(model_default)

8.7829             增持    -8.4829             中性
2.4511           2017    -3.7093             辞职
1.9163             增速    -3.4770             减持
1.8946             推荐    -2.6554           2016
1.7738            三季度    -2.2863           2015
1.6567           2018    -2.2855       20162018
1.4513             买入    -2.0567            上半年
1.3967             分别    -1.8135           1h17
1.3659             环比    -1.7969             16
1.2255             项目    -1.7378             平仓
1.2219             龙头    -1.6685             离职
1.1808           4q17    -1.6122           2q17
1.1654             18    -1.5751             快递
1.1356           1719    -1.4469            asp
1.1308             动力    -1.4461             冻结
1.1284             提升    -1.3439            一季度
1.1231             有望    -1.3235             t2
1.1088         201819    -1.3189            特高压
1.1086            进一步    -1.2642             持有
1.0888             持续    -1.2211             贷款
1.0841             熊猫    -1.1607        